# VacationPy
----

#### Dependencies and Setup

In [1]:
# dependencies and api key
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from api_keys import gkey

# urls
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# data paths
city_data_path = "../output_data/cities.csv"

In [2]:
# read in csv
city_data = pd.read_csv(city_data_path)

In [3]:
# gmaps configuration
gmaps.configure(api_key=gkey)

### Humidity Heatmap

In [4]:
# create function for heatmap
def heatmap(locations, weights):
    fig = gmaps.figure()
    heat_layer = gmaps.heatmap_layer(locations, weights=weights)
    fig.add_layer(heat_layer)
    return fig

In [5]:
# define locations and weights
locations = city_data[["Lat", "Lng"]]
weights = city_data["Humidity"]

# create and display the heatmap
humidity_heatmap = heatmap(locations, weights)
humidity_heatmap

Figure(layout=FigureLayout(height='420px'))

#### Weather Data

In [6]:
vacation_cities = city_data.dropna()

cold = vacation_cities.loc[vacation_cities["Max Temp"] < 65].index
vacation_cities = vacation_cities.drop(cold)

hot = vacation_cities.loc[vacation_cities["Max Temp"] > 85].index
vacation_cities = vacation_cities.drop(hot)

windy = vacation_cities.loc[vacation_cities["Wind Speed"] > 10].index
vacation_cities = vacation_cities.drop(windy)

cloudy = vacation_cities.loc[vacation_cities["Cloudiness"] > 0].index
vacation_cities = vacation_cities.drop(cloudy)

humid = vacation_cities.loc[vacation_cities["Humidity"] > 60].index
vacation_cities = vacation_cities.drop(humid)

hotel_df = vacation_cities

hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
59,Broken Hill,0,AU,1579993399,30,-31.95,141.43,80.67,5.98
194,Guerrero Negro,0,MX,1579993422,53,27.98,-114.06,67.24,4.61
202,Lovington,0,US,1579993424,20,32.94,-103.35,66.00,5.36
286,Kontagora,0,NG,1579993381,14,10.40,5.47,69.98,3.96
325,Acajutla,0,SV,1579993492,56,13.59,-89.83,82.00,0.89


In [7]:
responses = []
hotel_names = []

target_type = "lodging"
radius = 5000

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    coords = f"{lat},{lng}"
    params = {
        "location": coords,
        "types": target_type,
        "radius": radius,
        "key": gkey
    }
    response = requests.get(base_url, params).json()
    responses.append(response)
    hotel_names.append(response["results"][0]["name"])

hotel_df["Hotel Name"] = hotel_names

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
59,Broken Hill,0,AU,1579993399,30,-31.95,141.43,80.67,5.98,Daydream Motel & Cottages
194,Guerrero Negro,0,MX,1579993422,53,27.98,-114.06,67.24,4.61,"Casa Laguna, Bed & Breakfast"
202,Lovington,0,US,1579993424,20,32.94,-103.35,66.00,5.36,Buena vida rv parks
286,Kontagora,0,NG,1579993381,14,10.40,5.47,69.98,3.96,City Gate Inn
325,Acajutla,0,SV,1579993492,56,13.59,-89.83,82.00,0.89,Villavela Hotel Boutique
414,Rawson,0,AR,1579993523,37,-43.30,-65.10,76.14,6.69,Hotel Punta León
420,Jardim,0,BR,1579993525,57,-21.48,-56.14,74.70,1.99,Hotel Estância
449,Presidencia Roque Saenz Pena,0,AR,1579993539,43,-26.79,-60.44,79.27,1.91,Hotel Aconcagua
554,Tonota,0,BW,1579993577,56,-21.48,27.48,71.47,0.98,6 Ville Hotel


In [8]:
hotels_dict = {
    "Hotel Name": [],
    "Hotel Address": [],
    "Hotel Rating": [],
    "Lat": [],
    "Lng": []
}

for response in range(len(responses)):
    hotels_dict["Hotel Name"].append(responses[response]["results"][0]["name"])
    hotels_dict["Hotel Address"].append(responses[response]["results"][0]["vicinity"])
    hotels_dict["Hotel Rating"].append(responses[response]["results"][0]["rating"])
    hotels_dict["Lat"].append(responses[response]["results"][0]["geometry"]["location"]["lat"])
    hotels_dict["Lng"].append(responses[response]["results"][0]["geometry"]["location"]["lng"])


hotel_data = pd.DataFrame(hotels_dict)

hotel_data

,Hotel Name,Hotel Address,Hotel Rating,Lat,Lng
0,Daydream Motel & Cottages,"77 Argent Street, Broken Hill",3.6,-31.962251,141.459816
1,"Casa Laguna, Bed & Breakfast","Baja California, El Domino, Centro, Guerrero N...",4.7,27.959275,-114.055272
2,Buena vida rv parks,"901 South Love Street, Lovington",5.0,32.943945,-103.348042
3,City Gate Inn,"1 Federal Lowcost Housing Estate, Kontagora",3.7,10.401760,5.472697
4,Villavela Hotel Boutique,"Avenida Miramar, Acajutla",5.0,13.588713,-89.834214
5,Hotel Punta León,"Juan de la Piedra 210, Playa Union",4.0,-43.320090,-65.048961
6,Hotel Estância,"Rua Deolindo Peixoto, 2041 - Residencial Parqu...",4.5,-21.488142,-56.156865
7,Hotel Aconcagua,"Azcuénaga 102, Sáenz Peña",4.1,-26.795300,-60.445248
8,6 Ville Hotel,Tonota,3.3,-21.437306,27.474469


### Hotel Map

In [9]:
def hotel_map(fig, locations, hotel_marks, hover_text):
    marker_layer = gmaps.marker_layer(locations)
    symbol_layer = gmaps.symbol_layer(locations, hover_text=hover_text)
    fig.add_layer(marker_layer)
    fig.add_layer(symbol_layer)
    return fig

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_data[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
fig = humidity_heatmap
hover_text = hotel_names

# Display Map
final_map = hotel_map(fig, hotel_locations, hotel_info, hover_text)

final_map

Figure(layout=FigureLayout(height='420px'))